# 🧠 Entendimiento del problema de negocio

## ✅ Pasos generales en un problema de clasificación

1. **Entendimiento del problema** (selección de la métrica más adecuada)  
2. **Obtención de datos y primer contacto**  
3. **División en Train y Test**  
4. **MiniEDA**: Análisis del target, análisis bivariante, entendimiento de las features, selección de las mismas (si es necesario)  
5. **Preparación del dataset de Train**: Conversión de categóricas, tratamiento de numéricas  
6. **Selección e instanciación de modelos**. Baseline.  
7. **Comparación de modelos** (por validación cruzada o tras optimización de hiperparámetros)  
8. **Selección de modelo final**: Optimización de hiperparámetros  
9. **Equilibrado del train set**  
10. **Evaluación contra test**  
11. **Análisis de errores y acciones futuras**  
12. **Persistencia del modelo en disco**

---

## 🧩 Guía práctica: ¿Qué métrica usar según el problema?

### 1. Clasificación binaria

| Negocio te dice... | Qué es importante | Métrica principal | Métricas complementarias |
|--------------------|-------------------|-------------------|---------------------------|
| "Evita tratar a quien no lo necesita" | Minimizar FP | **Precision** | F1, matriz de confusión |
| "No podemos dejar sin tratar a un enfermo" | Minimizar FN | **Recall** | F1, curva PR |
| "Queremos equilibrio entre FP y FN" | Balance | **F1-score** | ROC-AUC, curva PR |
| "Queremos minimizar costes" | Costes específicos | **Custom loss** | Coste por predicción |

> ⚠️ **Evitar accuracy si hay desbalance**

---

### 2. Clasificación multiclase

| Escenario | Métricas clave |
|----------|----------------|
| Clases equilibradas | Accuracy, macro F1 |
| Clases desbalanceadas | Recall por clase, weighted F1 |
| Coste desigual por clase | Métricas ponderadas (confusión por clase) |

---

### 3. Regresión

| Negocio te dice... | Qué medir | Métrica recomendada | Alternativas |
|--------------------|-----------|----------------------|--------------|
| "Que no se equivoque mucho de media" | Error medio | **MAE** | MSE |
| "Castiga más los errores grandes" | Errores extremos | **RMSE** | MAE |
| "Nos importa el porcentaje de error" | Escala relativa | **MAPE** | MAE, RMSE |
| "Queremos saber si mejora la media" | Explicación global | **R²** | RMSE, MAE |

---

### 4. Agrupamiento (Clustering)

| Tipo de validación | Métrica recomendada |
|--------------------|---------------------|
| Interna (sin etiquetas reales) | Silhouette Score, Davies-Bouldin |
| Externa (con etiquetas reales) | ARI, NMI, Homogeneity |

---

### 5. Series temporales

| Requisito de negocio | Métrica recomendada |
|----------------------|---------------------|
| Error absoluto | MAE |
| Penalizar errores altos | RMSE |
| Comparación relativa | sMAPE, MASE |
| Evaluación visual | Gráfico real vs predicción |

---

## 📌 Esquema rápido de decisión

```markdown
# ¿Qué tipo de problema tengo?
- Binario → Precision / Recall / F1
- Multiclase → Accuracy / macro F1
- Regresión → MAE, RMSE, MAPE
- Clustering → Silhouette, ARI
- Series temporales → MAE, RMSE, sMAPE

# ¿Qué me pide negocio?
- ¿Evitar errores graves? → Precision / Recall
- ¿Evitar errores grandes? → RMSE
- ¿Medir error medio? → MAE
- ¿Medir % de error? → MAPE
- ¿Explicabilidad? → R²

# ¿Qué hago para comparar?
- Métrica principal + matriz de confusión o gráfico
- Validación cruzada si hay pocos datos


# 🧠 SPRINT 14: ML - Modelos No Supervisados II

## 🔻 Reducción de Dimensionalidad

### 🌌 Maldición de la Dimensionalidad

- Cada instancia de un dataset es un vector en un espacio de `n` dimensiones.
- A mayor número de features, mayor complejidad y **dispersión** de los datos.
- El número de muestras necesarias para representar correctamente la información **crece exponencialmente** con el número de dimensiones.

#### Consecuencias:
- Las instancias se alejan entre sí → más difícil detectar patrones.
- Riesgo de **overfitting**: nuevas observaciones se encuentran demasiado lejos de los datos originales.
- Modelos menos robustos y generalizables.

#### Solución parcial:
- Incrementar el conjunto de entrenamiento.
- Aplicar **reducción de dimensionalidad** para simplificar el espacio de representación.

---

## 🎯 Objetivos de la Reducción de Dimensionalidad

- Reducir la complejidad del modelo.
- Mejorar la **capacidad de generalización**.
- Disminuir el tiempo de procesamiento.
- Eliminar ruido e irrelevancia.
- Hacer más robustos los modelos ante datos nuevos.

---

## 🧰 Tipos de Reducción

### 1. Selección de Features (Feature Selection)

- Se **mantienen las variables originales**, seleccionando solo las más relevantes.
- Puede hacerse con métodos **supervisados o no supervisados**.
- Pros:
  - Interpretabilidad total.
- Contras:
  - Se pierde la información de las relaciones entre features descartadas.
  - Riesgo de no capturar combinaciones informativas.

### 2. Transformación de Features (Feature Extraction)

- Se **crean nuevas variables** a partir de combinaciones de las originales.
- Las nuevas variables pueden ser no interpretables directamente.
- Proceso de **transformación + selección**.
- Se busca condensar la información conservando la máxima varianza.

---

## 🧮 Técnicas de Transformación: Proyección

La proyección transforma los datos a un espacio de menor dimensión preservando la mayor cantidad posible de información.

### 📌 PCA (Análisis de Componentes Principales)

- Crea nuevas variables (componentes) como combinaciones lineales de las originales.
- Se ordenan por varianza explicada.
- Permite elegir cuántas componentes mantener según la varianza acumulada deseada.
- Muy útil para visualización en 2D o 3D y como paso previo a clustering.

```python
from sklearn.decomposition import PCA
pca = PCA(n_components=2)
X_pca = pca.fit_transform(X_scaled)
```

## 🧮 Introducción a PCA (Principal Component Analysis)

### 📌 ¿Qué es PCA?

El Análisis de Componentes Principales (PCA) es una técnica estadística de **proyección** que:

- Transforma un conjunto de variables originales correlacionadas en un conjunto nuevo de **componentes principales** no correlacionadas.
- Busca **reducir la dimensionalidad** de un dataset manteniendo la **mayor cantidad posible de varianza** (información).
- Es muy útil para simplificar espacios con muchas dimensiones, reduciendo ruido y facilitando el análisis.

---

### 🎯 Objetivo de PCA

> Dado un dataset con `n` individuos y `p` variables (X₁, X₂, ..., Xₚ), PCA intenta encontrar un número `z < p` de **factores subyacentes** que expliquen casi la misma información que las `p` variables originales.

Cada una de estas nuevas variables se llama **componente principal (CP)**.

---

### ⚙️ ¿Cómo funciona?

1. **Transformación**:
   A partir de `m` features originales (x₁, x₂, ..., xₘ), PCA genera **m componentes principales**, combinando las variables originales:

CP1 = c₁₀·z₀ + c₁₁·z₁ + c₁₂·z₂ + ... + c₁ₘ·zₘ


2. **Orden de importancia**:
- Las componentes están ordenadas por **varianza explicada**.
- `CP1` es la más informativa, `CP2` la segunda, y así sucesivamente.
- Se seleccionan las `n` primeras componentes que explican un **porcentaje deseado** de la varianza original.

3. **Reducción**:
- Cada nueva observación se representa por sus valores en las `n` componentes seleccionadas.
- Es decir, las features pasan a ser los valores proyectados en el nuevo espacio.

---

### 📊 ¿Qué representa la "información"?

En PCA, la **información** se asocia a la **varianza**:

- Se intenta **maximizar la varianza** capturada por cada componente.
- Se preservan también las **covarianzas** entre variables, reflejando la estructura interna del dataset.

> **Cuanta más varianza conserva un componente, más "informativo" es.**

---

### 📐 ¿Qué hace PCA exactamente?

- Identifica los **hiperplanos** del espacio de variables que **maximizan la varianza**.
- **Proyecta** los datos sobre esos hiperplanos, minimizando la pérdida de información.
- Reduce el número de dimensiones manteniendo la estructura de los datos.

---

### 🧠 ¿Para qué se utiliza PCA?

- 🔍 **Extracción de patrones**: permite detectar similitudes y diferencias en los datos.
- ✂️ **Reducción de dimensionalidad**: útil en datasets con cientos o miles de variables.
- 📉 **Visualización**: transforma datos de muchas dimensiones a 2D o 3D para facilitar su representación gráfica.
- ⚙️ **Preprocesamiento previo a clustering o modelos supervisados**: mejora eficiencia y reduce sobreajuste.

---

### 📌 Ejemplo práctico en código

```python
from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler

# Escalado de los datos
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

# Aplicación de PCA
pca = PCA(n_components=2)
X_pca = pca.fit_transform(X_scaled)

# Varianza explicada
print(pca.explained_variance_ratio_)
```

✅ Resumen: PCA es una técnica de transformación no supervisada que permite simplificar datasets complejos sin perder información esencial. Ideal para visualizar, reducir dimensionalidad y preparar datos para clustering o modelado.


## ⚙️ Algoritmo PCA

### 🧠 Conceptos matemáticos clave

---

#### 📊 Matriz de Covarianza

La **covarianza** mide en qué medida dos variables varían conjuntamente respecto a sus medias:

- Si `Cov(X, Y) > 0`: ambas variables tienden a aumentar juntas (relación positiva).
- Si `Cov(X, Y) < 0`: cuando una sube, la otra tiende a bajar (relación negativa).
- Si `Cov(X, Y) = 0`: no hay relación lineal entre las variables.

**Fórmula de la covarianza:**

\[
\text{Cov}(X, Y) = \frac{1}{n-1} \sum_{i=1}^{n} (X_i - \bar{X})(Y_i - \bar{Y})
\]

- La **matriz de covarianza** recoge todas las covarianzas posibles entre pares de variables.
- Es similar a la **matriz de correlación**, pero sin escalar los valores. La correlación está acotada entre -1 y 1, la covarianza no.

Ejemplo para un dataset con dos features `b₀` y `b₁`:

\[
\text{CovMatrix} =
\begin{bmatrix}
\text{Cov}(b_0, b_0) & \text{Cov}(b_0, b_1) \\
\text{Cov}(b_1, b_0) & \text{Cov}(b_1, b_1)
\end{bmatrix}
\]

---

#### 🧩 Autovalores y Autovectores

En álgebra lineal:

- Un **autovector** de una matriz es un vector que no cambia de dirección al multiplicarse por dicha matriz.
- El escalar asociado se llama **autovalor** (λ).

\[
A \cdot v = \lambda \cdot v
\]

Donde:
- `A` es una matriz cuadrada.
- `v` es un autovector (|v| = 1).
- `λ` es el autovalor (potencia explicativa).

**Propiedades:**
- La matriz debe ser cuadrada.
- Hay tantos autovalores como dimensiones tenga la matriz.
- Los autovectores son ortogonales entre sí.
- Cada autovalor indica cuánta **varianza** (información) aporta el autovector asociado.

En PCA:
- Los **autovectores** de la matriz de covarianza se convierten en los **componentes principales**.
- Los **autovalores** indican la **importancia relativa** de cada componente.

---

### 🧮 Pasos del algoritmo PCA

1. **Estandarizar los datos**  
   PCA es sensible a la escala → imprescindible escalar con `StandardScaler` o similar.

2. **Calcular la matriz de covarianza**  
   Analiza cómo varían conjuntamente las variables del dataset.

3. **Obtener autovalores y autovectores**  
   Se descompone la matriz de covarianza para obtener las **direcciones de máxima varianza**.

4. **Ordenar los componentes**  
   Los autovectores se ordenan según sus autovalores de mayor a menor → obtenemos los **componentes principales**.

5. **Construir el nuevo espacio**  
   Se seleccionan los `k` primeros componentes principales según la **varianza acumulada deseada**.

6. **Transformar el dataset**  
   Aplicamos los autovectores seleccionados al dataset original → obtenemos el dataset reducido.

---

### 🧭 Elección del número de componentes

- Seleccionamos cuántos componentes mantener según el porcentaje de **varianza explicada acumulada**.
- Si nos quedamos con todos los componentes, simplemente transformamos el dataset sin pérdida de información.
- **El objetivo es reducir**, no solo transformar, por lo que tiene sentido quedarse con los componentes más explicativos.

---

### ⚠️ Consideraciones importantes

- **Escalado de variables**  
  PCA maximiza la varianza → hay que asegurarse de que todas las variables estén en la misma escala.

- **Outliers**  
  La varianza es muy sensible a valores atípicos → estudiar y tratar los outliers antes de aplicar PCA.

- **Varianza explicada**  
  Permite saber cuánta información se pierde al reducir la dimensionalidad. Lo ideal es usar el menor número de componentes que expliquen un alto porcentaje (por ejemplo, 90-95%).

```python
# Ejemplo en código
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA

scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

pca = PCA(n_components=0.95)  # Mantener el 95% de la varianza
X_pca = pca.fit_transform(X_scaled)

print(pca.explained_variance_ratio_)  # Varianza explicada por componente
```
✅ Resumen:
El algoritmo PCA transforma el espacio de datos buscando las direcciones que maximizan la varianza, y permite comprimir datasets muy complejos en un espacio reducido con mínima pérdida de información.

La obtención de la PCA siguentes pasos:  
1. Escalado de las variables  
2. Instanciación PCA   
3. "Entrenamiento"  
4. Mostrar valores PCA [No siempre es necesario] en función de las features iniciales
5. Análisis de la varianza Explicada: Selección del número de componentes necesarias [Depende de la aplicación]  
6. Generación del dataset "PCAdo"